In [3]:
import numpy as np

In [61]:
# Sample exaclty k items from a population with marginal probabilities given by p (p sum to k).
# Returns a list of indices of selected items
# Source: https://projecteuclid.org/journals/annals-of-mathematical-statistics/volume-20/issue-3/On-the-Theory-of-Systematic-Sampling-II/10.1214/aoms/1177729988.full
def systematic_sampling(k, p):
    n = len(p)
    assert np.isclose(sum(p), k), "Marginal probabilities must sum to k"

    # Randomly permute order of items
    perm = np.random.permutation(n)
    p = [p[i] for i in perm]

    # Compute cumulative probabilities with S[0] = 0
    S = np.cumsum(p)
    S = np.insert(S, 0, 0)  # Now length n+1
    
    # Generate sorted sampling points 
    u = np.random.uniform(0, 1)
    sampling_points = [u + m for m in range(k)]
    
    # Select items with each point in [S[j], S[j+1])
    selected = []
    j = 0  # Pointer to current interval
    for point in sampling_points:
        # Advance pointer until we find S[j] > point
        while j < len(S) and S[j] <= point:
            j += 1
        selected.append(perm[j-1])  # Items are 1-indexed, so we subtract 1
    
    return selected

def verify_sampling(n, k, p, num_trials=10_000):
    counts = np.zeros(n)
    
    for _ in range(num_trials):
        sample = systematic_sampling(k, p)
        for item in sample:
            counts[item] += 1
            
    empirical_p = counts / num_trials  # Correct normalization
    return empirical_p

In [69]:
# Example usage with verification
n = 5000
k = 10
p = np.random.dirichlet(np.ones(n))
p = p / sum(p) * k  # Normalize to sum to k

sampled_p = verify_sampling(n, k, p)

In [70]:
np.max(sampled_p - p)

0.0026080047915283695

In [71]:
np.min(sampled_p - p)

-0.0025086028903444886